# Visualisation of single-cell expression data using PCA

In this lab you will use PCA to visualise some single\-cell gene expression data from Guo et al. "Resolution of Cell Fate Decisions Revealed by Single\-Cell Gene Expression Analysis from Zygote to Blastocyst" Developmental Cell, Volume 18, Issue 4, 20 April 2010, Pages 675\-685, available from http://dx.doi.org/10.1016/j.devcel.2010.02.012. The paper pdf is available in the handouts folder for Week 7 or on blackboard. 

The data are from a qPCR TaqMan array single cell experiment in mouse. The data is taken from the early stages of development when the Blastocyst is forming. At the 32 cell stage the data is already separated into the trophectoderm (TE) which goes onto form the placenta and the inner cellular mass (ICM). The ICM further differentiates into the epiblast (EPI)---which gives rise to the endoderm, mesoderm and ectoderm---and the primitive endoderm (PE) which develops into the amniotic sack. Guo et al selected 48 genes for expression measurement. They labelled the resulting cells and their labels are included as an aide to visualization.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import visualisation 
import plotly
import plotly.express as px
import plotly.graph_objs as go
import sklearn

#  Data summary

In [3]:
data = pd.read_csv('GuoData.csv', index_col=[0])
labels = data.index 
N, D = data.shape
print('Cells: %s, Genes: %s'%(N, D))

Cells: 437, Genes: 48


In [4]:
data

,Actb,Ahcy,Aqp3,Atp12a,Bmp4,Cdx2,Creb312,Cebpa,Dab2,DppaI,...,Sox2,Sall4,Sox17,Snail,Sox13,Tcfap2a,Tcfap2c,Tcf23,Utf1,Tspan8
1,-0.541050,1.203007,-1.030746,-1.064808,-0.494782,0.167143,1.369092,-1.083061,-0.668057,1.553758,...,1.351757,1.793476,-0.783185,1.408063,0.031991,0.351257,1.078982,-0.942981,-1.348892,1.051999
1,-0.680832,1.355306,-2.456375,-1.234350,-0.645494,-1.003868,1.207595,-1.208023,-0.800388,1.435306,...,1.363533,1.782172,-1.532477,1.361172,0.501715,-1.082362,0.930112,-1.064399,-1.469397,0.996275
1,-1.056038,1.280447,-2.046133,-1.439795,-0.828121,-0.983404,1.460032,-1.359447,-0.530701,1.340283,...,1.296802,1.567402,-3.194157,1.301777,0.445219,-0.031284,1.005767,-1.211529,-1.615421,0.651393
1,-0.732331,1.326911,-2.464234,-1.244323,-0.654359,-0.947023,1.265609,-1.215373,-0.765212,1.431401,...,1.684100,1.915556,-2.962515,1.349710,-1.875957,-1.699892,1.059458,-1.071541,-1.476485,0.699586
1,-0.629333,1.244308,-1.316815,-1.304162,-0.707552,-1.429070,0.895578,0.007785,-0.644606,1.381937,...,1.304653,1.761825,-1.265379,1.320533,0.609864,-0.413826,0.888624,-1.114394,-1.519017,0.798985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64 TE,0.981846,-0.652971,0.376022,1.414495,-0.372440,0.978892,0.636869,0.856056,0.767485,1.365015,...,-1.112031,-0.618734,0.344299,-0.952105,0.272503,1.453339,-0.299896,-0.844418,-1.251070,1.276398
64 TE,1.298196,-0.980800,0.767402,1.408511,-0.551521,0.644643,-0.926355,0.478230,0.739008,1.428797,...,-0.385848,-1.068621,-1.203923,-1.057349,0.064275,1.068977,-0.966150,-0.988691,-1.394258,1.154409
64 TE,0.716994,-0.787200,-0.020074,-1.226372,-0.638402,0.806083,-1.003183,1.200070,0.506173,1.331172,...,-0.185657,-0.686556,-0.175714,-1.108408,0.262818,0.877706,-0.270610,-1.058685,-1.463726,1.170975
64 TE,1.003917,-1.522880,0.289572,1.532177,-0.246551,0.603714,-0.656671,-0.877241,0.940018,1.345490,...,-0.911840,-1.226873,-1.071556,-0.878122,0.038448,1.324004,-0.731863,-0.742998,-1.150413,1.160433


In [5]:
W, scores, fracs = visualisation.do_pca(data)

scores = scores/abs(scores).max().max() 
scores.index = labels

W

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20
Actb,-0.047350,0.224099,-0.201776,-0.045059,0.073464,0.087576,-0.315455,-0.010362,-0.471079,0.040354,0.114998,0.082742,0.006810,0.098027,0.015964,0.007116,-0.083350,-0.013328,0.096225,-0.005968
Ahcy,0.165470,0.000027,-0.199850,-0.048614,0.252143,0.027637,-0.025472,0.114275,0.071682,-0.034983,-0.008191,-0.087918,-0.268319,-0.316035,-0.155110,0.202272,-0.001628,0.175557,-0.109996,0.056393
Aqp3,-0.022536,0.251628,-0.088326,-0.244430,-0.140909,-0.062961,-0.075672,0.024853,0.097900,-0.002928,0.085417,-0.044371,0.039974,-0.072969,-0.094152,-0.077926,0.173117,-0.037427,-0.062959,-0.217108
Atp12a,-0.128493,0.173022,-0.007638,0.019303,0.011942,0.062903,0.413335,-0.030453,-0.001614,-0.044863,-0.033282,0.020464,-0.179029,0.072591,-0.188801,-0.154189,0.027487,-0.020990,0.503163,-0.183544
Bmp4,0.156475,-0.074628,-0.026613,-0.151392,-0.089617,0.406868,0.140304,-0.091822,-0.078320,0.021328,0.087481,-0.063592,-0.244895,-0.086539,-0.243228,-0.064489,0.008971,-0.057244,-0.239360,-0.123866
Cdx2,-0.030195,0.249561,-0.047609,-0.046550,0.080276,0.029282,0.189084,0.267924,-0.020559,-0.016764,-0.104175,-0.287561,0.318421,0.112240,-0.162762,-0.014668,-0.071575,0.149446,-0.081224,0.522062
Creb312,0.139181,-0.145319,-0.035622,0.243798,0.158301,0.128237,0.007843,0.216564,-0.066889,0.084805,-0.118554,0.027423,-0.029298,0.182093,-0.018190,-0.237955,0.035050,0.000231,0.228507,0.140033
Cebpa,0.012392,0.202752,0.133496,0.000570,-0.290675,0.124453,-0.189945,0.198212,0.285393,0.101226,0.116618,-0.070413,-0.218769,0.043458,0.079114,0.084784,-0.311775,0.363089,0.388653,0.036638
Dab2,-0.067675,0.188677,-0.220877,0.215196,0.037822,0.063943,-0.035806,-0.072416,0.070053,-0.224344,0.083296,-0.033677,0.065606,-0.248908,-0.153128,-0.052384,0.183140,0.288388,-0.028840,-0.191924
DppaI,-0.203679,0.050437,0.164803,0.020537,0.131448,0.149462,0.073420,0.003098,0.012887,0.073889,-0.097929,-0.057425,-0.035667,-0.056790,-0.118205,0.038235,0.096604,-0.087026,0.046514,0.021217


In [6]:
x = np.arange(1,len(fracs)+1)
y = np.cumsum(fracs) # Cumulative sum of elements in the fracs array
fig = go.Figure(data=go.Scatter(x=x, y=y))
fig.update_layout(xaxis_title='Principal component', yaxis_title='Variance explained (cummulative)',
                 xaxis = dict(dtick = 1.0))
fig.show()

#11 principal components are required to explain 80% of the variance 

In [7]:
XPC = 'PC1' 
YPC = 'PC4' 
fig = px.scatter(scores, x=XPC, y=YPC, color=labels, hover_data=[XPC,YPC])
fig.update_traces(mode='markers', marker_line_width=1, marker_size=8)
fig.show()

#PC1 is the best at seperating the TE cells from 

In [8]:
XPC = 'PC1' 
YPC = 'PC3' 
fig = px.scatter(scores, x=XPC, y=YPC, color=labels, hover_data=[XPC,YPC])
fig.update_traces(mode='markers', marker_line_width=1, marker_size=8)
fig.show()

#PC1 and PC3 best seperate the 2 cell stage from the rest

In [9]:
scores

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20
1,0.127131,-0.090823,0.544151,0.204852,0.687869,-0.012922,-0.081237,0.047979,-0.084175,0.043963,0.020248,-0.106161,-0.044495,-0.094667,0.034374,0.054726,0.071990,0.019003,-0.062780,0.015346
1,0.128410,-0.198860,0.615488,0.045729,0.692622,-0.051111,-0.249154,-0.043865,0.012489,0.004201,-0.075999,-0.044638,-0.040979,-0.062788,-0.020408,0.043938,0.115949,0.048821,-0.033506,0.011041
1,0.031852,-0.272489,0.592303,0.064105,0.781658,-0.162601,-0.143698,0.025287,0.034053,0.072260,0.071474,0.019056,-0.038439,-0.195654,0.013567,0.035854,0.051603,0.021979,-0.014895,0.049758
1,0.089133,-0.290265,0.531210,0.069393,0.791632,-0.086518,-0.275670,-0.023174,0.087339,0.107435,0.088305,-0.145236,-0.005556,-0.150000,-0.017106,-0.078698,0.019101,-0.007501,0.045448,-0.039887
1,0.059269,-0.205119,0.585293,0.057503,0.607309,0.003797,-0.340001,0.034799,0.058659,-0.019408,-0.108267,-0.022060,-0.138996,-0.098421,-0.009981,0.091647,0.050793,0.020208,0.029476,-0.018893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64 TE,-0.555489,0.166184,0.010963,0.132251,-0.014288,0.165840,-0.000941,0.186958,-0.061225,0.041327,0.038190,-0.006394,0.007423,0.058622,-0.106540,0.075080,0.002103,-0.008350,0.039787,0.038421
64 TE,-0.598506,0.204619,0.015694,0.009824,0.026139,0.165949,0.003141,0.003867,-0.120237,0.071241,0.076808,0.067610,-0.025707,-0.136462,0.016904,-0.083829,-0.022717,-0.015971,0.055612,0.036983
64 TE,-0.570348,0.103520,0.159355,0.037452,-0.076801,0.185154,-0.266657,0.095726,-0.034770,-0.000621,-0.075356,-0.012318,0.052486,-0.088493,-0.036308,0.087524,0.002141,0.019156,-0.072462,0.095977
64 TE,-0.635901,0.090540,0.009666,0.070389,-0.038542,0.070669,0.042892,0.069634,-0.144213,0.052568,0.104043,0.062589,0.105851,-0.078457,-0.132584,-0.026549,0.038508,-0.067045,-0.005551,-0.006120


In [10]:
feature = 'Esrrb'
fig = visualisation.pca_biplot(W, scores, data, topN=10, XPC='PC1', YPC='PC2', feature=feature)
fig.show()

#To seperate the TE cell types I would use Esrrb as the lack of gene expression sepeates out these cell types from the rest of the groups